# <font color='light-blue' face='segoe ui'>PERMIT v. DENY Model</font>
Ameer Syedibrahim

#### PURPOSE:
This machine learning model is trained on a dataset containing images of hunting animals (targets which humans are permitted to shoot at) and crowds of people (targets which humans are not permitted to shoot at)
#### NOTE:
The model was trained using a 5-layer Convolutional Neural Network. All images are used from the ImageNet© Database.
Hardware limitations prevented the use of large datasets. Thus, the accuracy of the model in the detection hunting animals is relatively low since 2,700 was not sufficient for an accurate representation. However, the accuracy of the model in the detection of crowds people is relatively high

In [ ]:
#__________________________________________
import keras #machine learning
import numpy as np #math
import scipy.misc
import urllib,urllib.request
import cv2
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from resizeimage import resizeimage

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/train/permit/image(1).png')  # this is a PIL image
img = img.resize((150,150))
x = img_to_array(img)  # this is a Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 150, 150, 3)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='permit', save_format='jpg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('Weights/first_try_low_test.h5')  # always save your weights after training or during training

In [ ]:
model.load_weights('Weights/first_try.h5')

In [ ]:
def make_prediction(the_image):
    
    img = resizeimage.resize_cover(the_image, [150, 150])
    new_img = img_to_array(img)
    fin = new_img.reshape((1,) + new_img.shape)
    prediction = model.predict(fin)
    the_prediction = prediction[0][0]
    
    if the_prediction == 1.0:
        
        return "PERMIT"
    
    else:
        return "DENY"

## <font color='green' face='segoe ui'>PERMIT TEST</font>
289 new images of hunting animals were passed through the model

Only 110 / 289 were correctly identified as PERMIT

#### Accuracy: 38.1% ❌

In [ ]:
for i in range(289):
    rand = load_img('data/test/permit/image(' + str(i+1) + ').png')
    print(make_prediction(rand))

## <font color='red' face='segoe ui'>DENY TEST</font>

154 new images of people were passed through the model

All 154 / 154 were correctly identified as DENY

#### Accuracy: 100% ✔️

In [ ]:
for i in range(154):
    rand = load_img('data/test/deny/image(' + str(i+1) + ').png')
    print(make_prediction(rand))

## <font color='orange' face='segoe ui'>ADDITIONAL EXPERIMENTATION</font>

#### The low PERMIT Accuracy (36.8%) demanded further investigation

 _________________________________________________________

## <font color='purple' face='segoe ui'>EXTRA TEST - Bison</font>

387 new images of Bison were passed through the model from the same Image-Net source

229/387 were correctly identified as PERMIT

#### Accuracy: 59.2% 


#### <font color='red'>NOTE: Originally, 633 Bison images were passed for training</font>

In [ ]:
for i in range(387):
    rand = load_img('exp/bisontest_files/img(' + str(i+1) + ').png')
    print(make_prediction(rand))

## <font color='purple' face='segoe ui'>EXTRA TEST - Bear</font>

492 new images of Bear were passed through the model from the same Image-Net source

175/492 were correctly identified as PERMIT

#### Accuracy: 35.6% ❌

#### <font color='red'>NOTE: Originally, 499 Bear images were passed for training</font>

In [ ]:
for i in range(491):
    rand = load_img('exp/beartest_files/image(' + str(i+1) + ').png')
    print(make_prediction(rand))

## <font color='purple' face='segoe ui'>EXTRA TEST - Bighorn Sheep</font>

246 new images of Bighorn Sheep were passed through the model from the same Image-Net source

65/246 were correctly identified as PERMIT

#### Accuracy: 26.4% ❌
#### <font color='red'>NOTE: Originally, 388 Bighorn Sheep images were passed for training</font>

In [ ]:
for i in range(246):
    rand = load_img('exp/sheeptest_files/image(' + str(i+1) + ').png')
    print(make_prediction(rand))

### <font color='brown' face='segoe ui'>THE PATTERN</font>

The greater the quantity of images passed during training, the higher the percent accuracy of model. By experimenting in blocks of animals, this idea became evident.

#### <font color='red'>SOLUTION:
Significantly increase the number of training images for EACH hunting animal</font>


In [ ]:
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model1.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model1.add(Dense(64))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1))
model1.add(Activation('sigmoid'))

model1.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
batch_size1 = 16

# this is the augmentation configuration we will use for training
train_datagen1 = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen1 = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator1 = train_datagen1.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size1,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator1 = test_datagen1.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size1,
        class_mode='binary')

In [ ]:
model1.fit_generator(
        train_generator1,
        steps_per_epoch=2000 // batch_size1,
        epochs=50,
        validation_data=validation_generator1,
        validation_steps=800 // batch_size1)
model1.save_weights('Weights/second_try.h5')  # always save your weights after training or during training

In [ ]:
model1.load_weights('Weights/second_try.h5')

In [ ]:
def make_prediction1(the_image):
    
    img = resizeimage.resize_cover(the_image, [150, 150])
    new_img = img_to_array(img)
    fin = new_img.reshape((1,) + new_img.shape)
    prediction = model1.predict(fin)
    the_prediction = prediction[0][0]
    
    if the_prediction == 1.0:
        
        return "PERMIT"
    
    else:
        return "DENY"

## <font color='green' face='segoe ui'>PERMIT TEST 2.0</font>
289 new images of hunting animals were passed through the model

215 / 289 were correctly identified as PERMIT

#### Accuracy: 74.4%

In [ ]:
for i in range(289):
    rand = load_img('data/test/permit/image(' + str(i+1) + ').png')
    print(make_prediction1(rand))

## <font color='red' face='segoe ui'>DENY TEST 2.0</font>

154 new images of people were passed through the model

153 / 154 were correctly identified as DENY

#### Accuracy: 99.4% ✔️

In [ ]:
for i in range(154):
    rand = load_img('data/test/deny/image(' + str(i+1) + ').png')
    print(make_prediction1(rand))